In [1]:
import pandas as pd 
import numpy as np

## 데이터 불러오기

In [2]:
train = pd.read_csv('train.csv', encoding='UTF-8')
test = pd.read_csv('test.csv', encoding='UTF-8')

C:\Users\82103\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
train.head()

,CLNT_ID,SESS_ID,HITS_SEQ,PD_C,PD_ADD_NM,PD_BRA_NM,PD_BUY_AM,PD_BUY_CT,SESS_SEQ,SESS_DT,...,TOT_SESS_HR_V,DVC_CTG_NM,ZON_NM,CITY_NM,KWD_NM,SEARCH_CNT,PD_NM,CLAC1_NM,CLAC2_NM,CLAC3_NM
0,200251,8343839,211,643942,사이즈:75 / 1개,오르시떼,"28,000",1,30,20180514,...,"2,262",mobile,Gyeonggi-do,Seongnam-si,남아 파자마,5,남아동 코튼륀 긴소매 상하(55-75) OR17FBGE028055 - 60,속옷/양말/홈웨어,유아동속옷,유아동내의
1,200251,8343839,211,643942,사이즈:75 / 1개,오르시떼,"28,000",1,30,20180514,...,"2,262",mobile,Gyeonggi-do,Seongnam-si,주니어 파자마,4,남아동 코튼륀 긴소매 상하(55-75) OR17FBGE028055 - 60,속옷/양말/홈웨어,유아동속옷,유아동내의
2,200251,8343839,211,643942,사이즈:75 / 1개,오르시떼,"28,000",1,30,20180514,...,"2,262",mobile,Gyeonggi-do,Seongnam-si,파자마,2,남아동 코튼륀 긴소매 상하(55-75) OR17FBGE028055 - 60,속옷/양말/홈웨어,유아동속옷,유아동내의
3,200251,8343839,211,537989,사이즈:85 / 1개,오르시떼,"33,000",1,30,20180514,...,"2,262",mobile,Gyeonggi-do,Seongnam-si,남아 파자마,5,여아동 셔틀콕 긴소매 상하(80-85) OR18SGGA0040802 - 80,속옷/양말/홈웨어,유아동속옷,유아동내의
4,200251,8343839,211,537989,사이즈:85 / 1개,오르시떼,"33,000",1,30,20180514,...,"2,262",mobile,Gyeonggi-do,Seongnam-si,주니어 파자마,4,여아동 셔틀콕 긴소매 상하(80-85) OR18SGGA0040802 - 80,속옷/양말/홈웨어,유아동속옷,유아동내의


In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2393262 entries, 0 to 2393261
Data columns (total 21 columns):
 #   Column           Dtype  
---  ------           -----  
 0   CLNT_ID          int64  
 1   SESS_ID          int64  
 2   HITS_SEQ         int64  
 3   PD_C             int64  
 4   PD_ADD_NM        object 
 5   PD_BRA_NM        object 
 6   PD_BUY_AM        object 
 7   PD_BUY_CT        int64  
 8   SESS_SEQ         int64  
 9   SESS_DT          int64  
 10  TOT_PAG_VIEW_CT  float64
 11  TOT_SESS_HR_V    object 
 12  DVC_CTG_NM       object 
 13  ZON_NM           object 
 14  CITY_NM          object 
 15  KWD_NM           object 
 16  SEARCH_CNT       int64  
 17  PD_NM            object 
 18  CLAC1_NM         object 
 19  CLAC2_NM         object 
 20  CLAC3_NM         object 
dtypes: float64(1), int64(8), object(12)
memory usage: 383.4+ MB


In [1]:
plt.figure(figsize=(16, 6))
sns.heatmap(train.isnull().T, cbar=False)
plt.show()

NameError: name 'plt' is not defined

## 데이터 전처리

In [5]:
train['PD_BUY_AM']  = train['PD_BUY_AM'].map(lambda x: int(str(x).replace(',','')) )
train['PD_BUY_CT']  = train['PD_BUY_CT'].map(lambda x: int(str(x).replace(',','')) )
test['PD_BUY_AM']  = test['PD_BUY_AM'].map(lambda x: int(str(x).replace(',','')) )
test['PD_BUY_CT']  = test['PD_BUY_CT'].map(lambda x: int(str(x).replace(',','')) )

## feature 생성

In [6]:
features = []
features_te = []

**[총구매액, 구매건수, 평균구매액, 최대구매액]**

In [7]:
train['AMOUNT'] = train['PD_BUY_AM'] * train['PD_BUY_CT']
test['AMOUNT'] = test['PD_BUY_AM'] * test['PD_BUY_CT']

In [8]:
f = train.groupby('CLNT_ID')['AMOUNT'].agg([('최소구매액', np.min),
                                            ('중위수', np.median),
                                            ('구매건수',np.size),
                                            ('총구매액',np.sum),
                                            ('평균구매액', lambda x : np.round(np.mean(x))),
                                            ('최대구매액', np.max)]).reset_index()
features.append(f);display(f)

f_te = test.groupby('CLNT_ID')['AMOUNT'].agg([('최소구매액', np.min),
                                            ('중위수', np.median),
                                            ('총구매액',np.sum),
                                            ('평균구매액', lambda x : np.round(np.mean(x))),
                                            ('최대구매액', np.max)]).reset_index()
features_te.append(f_te) ; display(f_te)

,CLNT_ID,최소구매액,중위수,구매건수,총구매액,평균구매액,최대구매액
0,1,39000,92000,16,1276000,79750,99000
1,4,12800,12800,6,125200,20867,37000
2,5,11000,42000,12,380000,31667,42000
3,6,8000,18000,7,148900,21271,44900
4,7,20600,29400,16,457900,28619,40000
...,...,...,...,...,...,...,...
184167,263099,7000,7000,112,973000,8688,22000
184168,263100,9500,11200,2,22400,11200,12900
184169,263101,33900,33900,4,135600,33900,33900
184170,263102,1080,1080,15,59400,3960,9720


,CLNT_ID,최소구매액,중위수,총구매액,평균구매액,최대구매액
0,0,5500,43250,86500,43250,81000
1,2,20000,45900,560000,50909,62100
2,3,102400,106400,851200,106400,110400
3,16,9500,9500,9500,9500,9500
4,17,2900,2900,5800,2900,2900
...,...,...,...,...,...,...
78927,263083,39100,39100,117300,39100,39100
78928,263085,18600,35000,1562720,40070,74500
78929,263086,8900,36000,1836300,41734,89000
78930,263094,10000,10000,10000,10000,10000


**[주말방문비율]**

In [9]:
train['date'] = pd.to_datetime(train['SESS_DT'], format= '%Y%m%d')
test['date'] = pd.to_datetime(test['SESS_DT'], format= '%Y%m%d')

In [10]:
f = train.groupby('CLNT_ID')['date'].agg([
    ('주말방문비율', lambda x: np.mean(x.dt.dayofweek>4))]).reset_index()
features.append(f); display(f)

f_te = test.groupby('CLNT_ID')['date'].agg([
    ('주말방문비율', lambda x: np.mean(x.dt.dayofweek>4))]).reset_index()
features_te.append(f_te);display(f_te)

,CLNT_ID,주말방문비율
0,1,0.125000
1,4,0.333333
2,5,0.000000
3,6,0.857143
4,7,0.312500
...,...,...
184167,263099,0.000000
184168,263100,0.500000
184169,263101,0.000000
184170,263102,0.000000


,CLNT_ID,주말방문비율
0,0,1.000000
1,2,0.000000
2,3,1.000000
3,16,1.000000
4,17,1.000000
...,...,...
78927,263083,0.000000
78928,263085,0.333333
78929,263086,0.000000
78930,263094,0.000000


**[계절방문비율]**

In [11]:
f = train.groupby('CLNT_ID')['date'].agg([
    ('봄-구매비율', lambda x: np.mean( x.dt.month.isin([3,4,5]))),
    ('여름-구매비율', lambda x: np.mean( x.dt.month.isin([6,7,8]))),
    ('가을-구매비율', lambda x: np.mean(x.dt.month.isin([9,10,11]))),
    ('겨울-구매비율', lambda x: np.mean( x.dt.month.isin([1,2,12])))
]).reset_index()
features.append(f); f

,CLNT_ID,봄-구매비율,여름-구매비율,가을-구매비율,겨울-구매비율
0,1,0.375000,0.625000,0.0,0.0
1,4,0.666667,0.333333,0.0,0.0
2,5,0.000000,0.000000,1.0,0.0
3,6,1.000000,0.000000,0.0,0.0
4,7,0.437500,0.562500,0.0,0.0
...,...,...,...,...,...
184167,263099,0.000000,1.000000,0.0,0.0
184168,263100,0.500000,0.500000,0.0,0.0
184169,263101,1.000000,0.000000,0.0,0.0
184170,263102,1.000000,0.000000,0.0,0.0


In [12]:
f_te = test.groupby('CLNT_ID')['date'].agg([
    ('봄-구매비율', lambda x: np.mean( x.dt.month.isin([3,4,5]))),
    ('여름-구매비율', lambda x: np.mean( x.dt.month.isin([6,7,8]))),
    ('가을-구매비율', lambda x: np.mean(x.dt.month.isin([9,10,11]))),
    ('겨울-구매비율', lambda x: np.mean( x.dt.month.isin([1,2,12])))
]).reset_index()
features_te.append(f_te); f_te

,CLNT_ID,봄-구매비율,여름-구매비율,가을-구매비율,겨울-구매비율
0,0,0.000000,1.000000,0.000000,0.0
1,2,0.000000,0.909091,0.090909,0.0
2,3,0.000000,0.000000,1.000000,0.0
3,16,0.000000,0.000000,1.000000,0.0
4,17,0.000000,1.000000,0.000000,0.0
...,...,...,...,...,...
78927,263083,0.000000,1.000000,0.000000,0.0
78928,263085,0.461538,0.538462,0.000000,0.0
78929,263086,0.227273,0.772727,0.000000,0.0
78930,263094,1.000000,0.000000,0.000000,0.0


## numeric feature 추가 생성

In [13]:
f = train.groupby('CLNT_ID')['date'].agg([
    ('봄-구매비율', lambda x: np.mean( x.dt.month.isin([3,4,5]))),
    ('여름-구매비율', lambda x: np.mean( x.dt.month.isin([6,7,8]))),
    ('가을-구매비율', lambda x: np.mean(x.dt.month.isin([9,10,11]))),
    ('겨울-구매비율', lambda x: np.mean( x.dt.month.isin([1,2,12])))
]).reset_index()
features.append(f); f

,CLNT_ID,봄-구매비율,여름-구매비율,가을-구매비율,겨울-구매비율
0,1,0.375000,0.625000,0.0,0.0
1,4,0.666667,0.333333,0.0,0.0
2,5,0.000000,0.000000,1.0,0.0
3,6,1.000000,0.000000,0.0,0.0
4,7,0.437500,0.562500,0.0,0.0
...,...,...,...,...,...
184167,263099,0.000000,1.000000,0.0,0.0
184168,263100,0.500000,0.500000,0.0,0.0
184169,263101,1.000000,0.000000,0.0,0.0
184170,263102,1.000000,0.000000,0.0,0.0


In [14]:
#여러가지 feature를 만들어보세요!


## data 생성

In [15]:
X_train = pd.DataFrame({'CLNT_ID': train.CLNT_ID.unique()})
for f in features :
    X_train = pd.merge(X_train, f, how='left')
    
X_train = X_train.fillna(0)

In [16]:
X_test = pd.DataFrame({'CLNT_ID': test.CLNT_ID.unique()})
for f in features_te :
    X_test = pd.merge(X_test, f, how='left')
    
X_test = X_test.fillna(0)

In [17]:
X_train['충동구매비율']=X_train['최대구매액']/X_train['총구매액']

In [18]:
X_train['구매건수당총구매액']=X_train['구매건수']/X_train['총구매액']

In [28]:
goods_count = pd.pivot_table(index='CLNT_ID',columns='CLAC1_NM',values='AMOUNT',aggfunc=np.size, fill_value=0, data=train.query("AMOUNT>0")[['CLNT_ID','CLAC1_NM','AMOUNT']]).reset_index()

In [29]:
X_train=pd.merge(X_train,goods_count,on='CLNT_ID')

In [ ]:
display(X_train)
display(X_test)